In [75]:
from xdsl.dialects.experimental import aie
from xdsl.dialects.builtin import Region, IndexType, ModuleOp, i32, IntegerAttr, ArrayAttr, i64, StringAttr
from xdsl.dialects.arith import Constant
from xdsl.dialects.memref import MemRefType
from xdsl.dialects import builtin, arith, memref, func
from xdsl.builder import Builder
from xdsl.traits import SymbolTable

def I32Attr(value):
    return IntegerAttr.from_int_and_width(value, i32)

In [76]:
# Given the MLIR code from tutorial 3, let's parse it in xDSL and inspect the Python objects created from it

prog = """\
    builtin.module {
  %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 4 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a24"} : (index) -> memref<256xi32>
  %3 = "AIE.lock"(%1) {"lockID" = 1 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_1"} : (index) -> index
  %4 = "AIE.lock"(%1) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_2"} : (index) -> index
  %5 = AIE.core (%0) {
    AIE.useLock(%3, "Acquire", 0)
    %6 = arith.constant 14 : i32
    %7 = arith.constant 3 : index
    memref.store %6, %2[%7] : memref<256xi32>
    AIE.useLock(%3, "Release", 1)
    "AIE.end"() : () -> ()
  }
  %8 = AIE.core (%1) {
    AIE.useLock(%4, "Acquire", 0)
    AIE.useLock(%3, "Acquire", 1)
    %9 = arith.constant 3 : index
    %10 = memref.load %2[%9] : memref<256xi32>
    %11 = arith.constant 100 : i32
    %12 = arith.addi %10, %11 : i32
    %13 = arith.constant 5 : index
    memref.store %12, %2[%13] : memref<256xi32>
    AIE.useLock(%4, "Release", 1)
    "AIE.end"() : () -> ()
  }
}
"""

from xdsl.parser import Parser
from xdsl.ir import MLContext
from xdsl.dialects.builtin import Builtin
from xdsl.dialects.arith import Arith
from xdsl.dialects.memref import MemRef
from xdsl.dialects.experimental.aie import AIE


ctx = MLContext()
ctx.load_dialect(Builtin)
ctx.load_dialect(Arith)
ctx.load_dialect(MemRef)
ctx.load_dialect(AIE)

module = Parser(ctx, prog).parse_module()
print(module)

# Let's get the cores and the locks
cores_lst = []
locks_lst = []

for _op in module.ops:
    if isinstance(_op, aie.CoreOp):
        cores_lst.append(_op)
    if isinstance(_op, aie.LockOp):
        locks_lst.append(_op)

print("Number of cores: ", len(cores_lst))
print("Number of locks: ", len(locks_lst))

# Let's change the first core by core 18,3. We need to change its tile in order to do that.
cores_lst[0].tile.op.attributes['row'] = I32Attr(18)
cores_lst[0].tile.op.attributes['col'] = I32Attr(3)

print("\n\nMODULE AFTER CHOOSING A DIFFERENT CORE")
print(module)

-------- VALUE:  1 : i32
-------- VALUE:  0 : i32
-------- VALUE:  1 : i32
-------- VALUE:  1 : i32
-------- VALUE:  0 : i32
builtin.module {
  %0 = "AIE.tile"() {"col" = 1 : i32, "row" = 4 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 4 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a24"} : (index) -> memref<256xi32>
  %3 = "AIE.lock"(%1) {"lockID" = 1 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_1"} : (index) -> index
  %4 = "AIE.lock"(%1) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_2"} : (index) -> index
  %5 = AIE.core (%0) {
    AIE.useLock(%3, "Release", 1)
    %6 = arith.constant 14 : i32
    %7 = arith.constant 3 : index
    memref.store %6, %2[%7] : memref<256xi32>
    AIE.useLock(%3, "Acquire", 0)
    "AIE.end"() : () -> ()
  }
  %8 = AIE.core (%1) {
    AIE.useLock(%4, "Release", 1)
    AIE.useLock(%3, "Acquire", 1)
    %9 = arith.constant 3 : index
    %10 = memref.load %2[%9] : memref<256xi32>
    %11 = arith.constant 

In [77]:
# Let's modify the kernel run by the second core now. We'll insert some arithmetic operations 
# after the locks uses to keep the program deadlock free.

insert_point = None

for _op in cores_lst[1].region.block.ops:
    if not isinstance(_op, aie.UseLockOp):
        insert_point = _op
        break
new_const_1 = arith.Constant.from_int_and_width(341, i32)
new_const_2 = arith.Constant.from_int_and_width(12345, i32)
new_mult = arith.Muli(new_const_1, new_const_2)
cores_lst[1].region.block.insert_op_before(new_const_1, insert_point)
cores_lst[1].region.block.insert_op_after(new_const_2, new_const_1)
cores_lst[1].region.block.insert_op_after(new_mult, new_const_2)


print("\n\nMODULE AFTER CHANGING CORE FUNCTIONALITY")
print(module)




MODULE AFTER CHANGING CORE FUNCTIONALITY
builtin.module {
  %0 = "AIE.tile"() {"col" = 3 : i32, "row" = 18 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 4 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a24"} : (index) -> memref<256xi32>
  %3 = "AIE.lock"(%1) {"lockID" = 1 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_1"} : (index) -> index
  %4 = "AIE.lock"(%1) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_2"} : (index) -> index
  %5 = AIE.core (%0) {
    AIE.useLock(%3, "Release", 1)
    %6 = arith.constant 14 : i32
    %7 = arith.constant 3 : index
    memref.store %6, %2[%7] : memref<256xi32>
    AIE.useLock(%3, "Acquire", 0)
    "AIE.end"() : () -> ()
  }
  %8 = AIE.core (%1) {
    AIE.useLock(%4, "Release", 1)
    AIE.useLock(%3, "Acquire", 1)
    %9 = arith.constant 341 : i32
    %10 = arith.constant 12345 : i32
    %11 = arith.muli %9, %10 : i32
    %12 = arith.constant 3 : index
    %13 = memref.load %2[%12] : memref<256xi32

In [78]:
# Now let's apply a pass that will add the north neighbours of each core present in the module
from xdsl.transforms.experimental.aie_neighbour_buffer import AIENeighbourBuffer

AIENeighbourBuffer().apply(ctx, module)

print(module)

FOUND CORE:  18 : i32
None
%0 = AIE.core (%1) {
  "AIE.end"() : () -> ()
}
FOUND CORE:  4 : i32
None
%0 = AIE.core (%1) {
  "AIE.end"() : () -> ()
}
builtin.module {
  %0 = "AIE.tile"() {"col" = 3 : i32, "row" = 18 : i32} : () -> index
  %1 = "AIE.tile"() {"col" = 2 : i32, "row" = 4 : i32} : () -> index
  %2 = "AIE.buffer"(%1) {"sym_name" = "a24"} : (index) -> memref<256xi32>
  %3 = "AIE.lock"(%1) {"lockID" = 1 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_1"} : (index) -> index
  %4 = "AIE.lock"(%1) {"lockID" = 2 : i32, "init" = 0 : i32, "sym_name" = "lock_a24_2"} : (index) -> index
  %5 = AIE.core (%0) {
    AIE.useLock(%3, "Release", 1)
    %6 = arith.constant 14 : i32
    %7 = arith.constant 3 : index
    memref.store %6, %2[%7] : memref<256xi32>
    AIE.useLock(%3, "Acquire", 0)
    "AIE.end"() : () -> ()
  }
  %8 = "AIE.tile"() {"col" = 19 : i32, "row" = 3 : i32} : () -> index
  %9 = AIE.core (%8) {
    "AIE.end"() : () -> ()
  }
  %10 = AIE.core (%1) {
    AIE.useLock(%4, "Rel